In [1]:
import datetime
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser, PydanticToolsParser
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI

In [2]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a viral Instagram influencer assistant tasked with writing excellent, catchy Instagram post captions.
            Current time : {time}
            1.{first_instruction}
            2.Critique your answer. Be severe to maximize improvements.
            3.Recommend search queries to research information and improve your answer.
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Generate a caption for users query above using the required format.")
    ]
).partial(time = lambda : datetime.datetime.now().isoformat())

In [3]:
llm = AzureChatOpenAI(
        azure_deployment="gpt-4o-mini",  # or your deployment
        api_version="2023-03-15-preview",  # or your api version
        temperature=0)

In [4]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field


class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")
    
    
class AnQ(BaseModel):
    answer : str = Field(description="~50 word short caption to the user query.")
    reflection : Reflection = Field(description="Your reflection on the initial caption.")
    search_queries : List[str] = Field(
        description = "1-3 search queries for researching improvements to address the critique of your current answer."
    )
    

C:\Users\vinay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
parser = JsonOutputToolsParser(return_id=True)
parser_pydantic = PydanticToolsParser(tools=[AnQ])

In [6]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction = "Provide a short ~50 word caption."
)

first_responder = first_responder_prompt_template| llm.bind_tools(
    tools=[AnQ], tool_choice="AnQ"
)



In [7]:
human_msg = HumanMessage(
    content="write caption for Goa vacation pics"
)

chain = first_responder | parser_pydantic

res = chain.invoke(input = {"messages":[human_msg]})
print(res)

[AnQ(answer='"Sun, sand, and sea – Goa, you have my heart! 🌊✨ From vibrant markets to serene beaches, every moment is a postcard memory. Can’t wait to share more of this paradise with you all! #GoaDiaries #BeachVibes"', reflection=Reflection(missing='The caption lacks a personal touch or specific experiences that could make it more relatable and engaging.', superfluous='The use of multiple hashtags could be streamlined to focus on the most relevant ones.'), search_queries=['how to write engaging Instagram captions', 'best practices for Instagram hashtags', 'tips for personalizing travel captions'])]


In [8]:
class ReviseAns(AnQ):
    answer : str = Field(description="~50 word short answer to the question.")
    reflection : Reflection = Field(description="Your reflection on the initial answer.")
    search_queries : List[str] = Field(
        description = "1-3 search queries for researching improvements to address the critique of your current answer."
    )

In [9]:
revise_instruction = """
                    You are a viral Instagram influencer grading a caption for a Instagram post. Generate critique and recommendation for the user's caption.
                    Always provide detailed recommendations, including requests for lengths, virality, style, etc. and make sure it is not more than 250 words.
                    """

revisor = actor_prompt_template.partial(
    first_instruction = revise_instruction
)| llm.bind_tools(
    tools=[ReviseAns], tool_choice="ReviseAns"
)

In [10]:
from langchain_core.messages import BaseMessage, ToolMessage, AIMessage
from langgraph.prebuilt import ToolInvocation, ToolExecutor
  

answer = AnQ(
    answer="",
    reflection=Reflection(missing="", superfluous=""),
    search_queries=[
        'best Instagram captions for travel', 'how to write engaging travel captions', 'Goa travel tips and experiences'
    ],
    id = "call_xyz",
)

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper()
tavily_tools = TavilySearchResults(api_wrapper =search, max_results = 5)

tool_executor = ToolExecutor([tavily_tools])

C:\Users\vinay\AppData\Local\Temp\ipykernel_21508\3995445223.py:7: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor([tavily_tools])


In [12]:
from collections import defaultdict
import json

def execute_tools(state : List[BaseMessage]) -> List[ToolMessage]:
    tools_invocation : AIMessage = state[-1]
    parsed_tool_calls = parser.invoke(tools_invocation)
    
    ids = []
    tool_invocations = []
    
    for parsed_call in parsed_tool_calls:
        for query in parsed_call["args"]["search_queries"]:
            tool_invocations.append(ToolInvocation(
                tool = "tavily_search_results_json",
                tool_input = query,
            ))
            ids.append(parsed_call["id"]) 
            
    outputs = tool_executor.batch(tool_invocations)
    outputs_map = defaultdict(dict)
    for id_, output, invocation in zip(ids, outputs, tool_invocations):
        outputs_map[id_][invocation.tool_input] = output
        
    tool_messages = []
    for id_, mapped_output in outputs_map.items():
        tool_messages.append(ToolMessage(content = json.dumps(mapped_output), tool_call_id=id_))

    return tool_messages

In [13]:
raw_res = execute_tools(
    state=[
        human_msg,
        AIMessage(
            content = "",
            tool_calls = [
                {
                    "name" : AnQ.__name__,
                    "args" : answer.dict(),
                    "id" : "call_xyz"
                }
            ]
        )
    ]
)

C:\Users\vinay\AppData\Local\Temp\ipykernel_21508\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(


In [14]:
from pprint import pprint

raw_res

[ToolMessage(content='{"best Instagram captions for travel": [{"url": "https://www.thediaryofanomad.com/best-travel-captions-for-instagram/", "content": "Here are some more best Instagram captions for traveling \\u2014 short, catchy, and classic sayings that will inspire wanderlust. These IG travel captions are also perfect for beach and vacation shots. My favorite thing is to go where I\'ve never been. Born to roam. Everywhere is home. Keep calm and travel on."}, {"url": "https://www.adamenfroy.com/travel-instagram-captions", "content": "Here are the top adventure travel captions to use in your Instagram profile as an adventure seeker: Chasing horizons and dreams. \\ud83c\\udf04. Adventures are the best way to learn. \\ud83d\\udcda. Wander where the Wi-Fi is weak. \\ud83c\\udf32. Seek adventures that open your mind. \\ud83d\\udd13."}, {"url": "https://www.adventureinyou.com/travel-tips/travel-captions-for-instagram/", "content": "If you want to keep it short and sweet with your travel

In [15]:
from typing import List
from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph


In [16]:
MAX_ITERATIONS=2
builder = MessageGraph()
builder.add_node("draft", first_responder)
builder.add_node("execute_tools",execute_tools)
builder.add_node("revise", revisor)
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")


In [17]:
def event_loop(state:List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"

In [18]:
builder.add_conditional_edges("revise", event_loop)
builder.set_entry_point("draft")

In [19]:
from mermaid import Mermaid
graph = builder.compile()
Mermaid(graph.get_graph().draw_mermaid())

In [25]:
res = graph.invoke("Write a instagram post caption in Marathi language for Maharashtrian Jewellery brand called 'PNG Jewellers' for Antique and Traditional ornaments")

C:\Users\vinay\AppData\Local\Temp\ipykernel_21508\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(
C:\Users\vinay\AppData\Local\Temp\ipykernel_21508\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(
C:\Users\vinay\AppData\Local\Temp\ipykernel_21508\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(


In [26]:
# print("Whole Result Object -", res,"--------------------\n\n\n")
print(res[-1].tool_calls[0]["args"]["answer"])

✨ प्राचीन आणि पारंपरिक दागिन्यांची जादू! PNG Jewellers मध्ये आपल्या संस्कृतीचा अभिमान व्यक्त करा. प्रत्येक तुकडा एक कथा सांगतो, आपल्या सौंदर्याला एक अनोखा स्पर्श देतो. आमच्या संग्रहात भेट द्या आणि आपल्या खास क्षणांना अधिक खास बनवा! 💖 #PNGJewellers #AntiqueJewellery #MaharashtrianTradition
